# Welcome to the PyTorch Sentiment Analysis Exercise

This exercise will cover some key concepts in natural language machine learning, including:

* The preparation of text datasets
* The construction of neural net models for natural language tasks
* The use of *recurrent neural network layers,* which are commonly used in problems involving sequences, including natural language tasks.

## Notes on Using This Notebook

* Code will be provided for boilerplate tasks; in other places, you will need to fill in code to complete the exercise. Cells you need to fill in will be flagged with the **Exercise** heading.
* The code cells are, in general, meant to be run in order. If you think a code cell should be working, but it isn't, verify that all previous cells were run - the cell you're having trouble with may depend on a variable or file that is created in a previous cell.
* Class names and other text normally meant for consumption by a computer will be rendered in a `monospace font`. This will hopefully reduce confusion between, e.g., the word "dataset" referring to the concept of a cohesive body of data, and the class name `Dataset` referring to the related PyTorch class.

### Do This Now:

The cell below downloads and unzips the dataset we'll be using for this exercise. The dataset isn't huge, but it may take a minute to download, so **please uncomment and execute the following code cell now** to get the process started. (The commented lines are there to prevent the download triggering accidentally, so you may wish to replace them afterward.)

In [0]:
# !curl -0 https://s3-us-west-1.amazonaws.com/pytorch-course-datasets/sentiment-analysis-on-movie-reviews.zip > reviews.zip
# !unzip reviews.zip
# !mkdir models

## Introduction

This exercise is based on the Kaggle competition, [Sentiment Analysis on Movie Reviews](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/overview). The goal is to accurately classify movie reviews based on how positively they speak of the movie being reviewed.

### The Training Dataset

Let's have a look at the input data. In `train.tsv`, you should see a file with four fields: A phrase ID, a sentence ID, a phrase, and a sentiment score. (The score runs from 0 to 4, with higher numbers indicating more positive sentiment.) You should see the same content in `test.tsv`, but without the scores.

### The Approach

There are many approaches to Natural Language Processing (NLP) using deep learning, and they will often include a variety of common neural network constructs, including fully-connected layers, recurrent neural networks (RNNs), convnets, embeddings, and dropouts. For this exercise, we'll be guiding you through constructing a simple model using word embeddings and RNNs, and pointing out directions for further enhancement and exploration.

### The Final Step

The test dataset is a separate, unlabeled collection of phrases from movie reviews. The final step in today's exercise will be to use your model to classify the unlabeled phrases. You will export your predictions to a file and upload them to the Kaggle site to receive a final accuracy score.

## Getting Started

In the cells below, we will:
* ...import some necessary modules
* ...define a few boilerplate helper functions
* ...create a global device handle for our CPU or GPU device

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import data       # NLP data utilities

import random
import time                      # timestamping messages and saved models
import os                        # for filesystem methods

In [0]:
def tlog(msg):
    print('{}   {}'.format(time.asctime(), msg))

def count_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def count_correct(guesses, labels):
    return (guesses == labels).float().sum()

def save_model(model, epoch):
    savefile = "{}-e{}-{}.pt".format('pytorch-sentiment', epoch, int(time.time()))
    tlog('Saving model {}'.format(savefile))
    path = os.path.join('models', savefile)
    # recommended way from https://pytorch.org/docs/stable/notes/serialization.html
    torch.save(model.state_dict(), path)
    return savefile

In [0]:
# global device handle
if not torch.cuda.is_available():
    device = torch.device('cpu')
    print('*** GPU not available - running on CPU. ***')
else:
    device = torch.device('cuda')
    print('GPU ready to go!')

## Setting Up the Training Data

Setting up natural language datasets in PyTorch is a bit different than setting up numeric time series or image data. For this reason, the PyTorch domain library `torchtext` includes a number of utilities that help with handling data for NLP tasks.

### `Field` Objects

NLP datasets are often presented in a structured form, such as a CSV or TSV file, where individual fields can be identified. To ease consumption of this data, `torchtext` offers a `Field` class, and related subclasses, as well as a `TabularDataset` class. The `Field` classes let you define datatypes and transformations (such as tokenization) on a field, and the `TabularDataset` takes a list of fields together with information about the file and its format, to parse the file into an internal representation that we'll find convenient. (See lines 10-26 in the code cell below.)

For this iteration of the exercise, we will skip the first two fields in the dataset, and create a single field for each phrase to be analyzed, and a field for the sentiment score associated with that field.

### Vocabularies and Embedding

The standard way of representing words in a natural language vocabulary is with **"one-hot" vectors** (known as *unit vectors* in other mathematical contexts). For a three-word vocabulary, this would look like:

```
word0 = torch.tensor([1, 0, 0])
word1 = torch.tensor([0, 1, 0])
word2 = torch.tensor([0, 0, 1])
```

Native speakers of most languages know 15,000 to 20,000 words; you can see how the above representation would consume untenable amounts of memory for large datasets. For this reason, PyTorch represents one-hot vectors by storing a single integer, which is the *index* of the single "hot" entry of the word vector.

Even with this optimization, the one-hot representation doesn't capture interesting features of the vocabulary, such as the semantic closeness of word pairs like "fantastic" and "amazing". The vocabulary vector space can be reduced further by *embedding* the vocabulary in a lower-dimensional space, which has the effect of reducing the complexity of the space in which we're trying to find ad descend gradients while capturing some of these relationships within the vocabulary.

You can create and train your own embedding using the `torch.nn.Embedding` layer, or you can use one of the pre-trained embeddings made available through `torchtext`. Below, we make use of the GloVe word embedding when we build our vocabulary for the `phrase` field in line 29.

In [0]:
# dataset constants
VOCAB_SIZE = 15000 # max size of vocabulary
VOCAB_VECTORS = "glove.6B.100d" # Stanford NLP GloVe (global vectors) for word rep
BATCH_SIZE = 64



def get_data():
    tlog('Preparing data...')
    phrases_fieldspec = data.Field(include_lengths=True, tokenize='spacy')
    labels_fieldspec = data.LabelField(dtype=torch.int64, sequential=False)

    fields = [
        ('SKIP_phrase_id', None),
        ('SKIP_sentence_id', None),
        ('phrases', phrases_fieldspec),
        ('labels', labels_fieldspec)
    ]


    train_data = data.TabularDataset(
        'train.tsv', # path to file
        'TSV', # file format
        fields,
        skip_header = True # we have a header row
    )
    
    train_data, eval_data = train_data.split()
    phrases_fieldspec.build_vocab(train_data, max_size=VOCAB_SIZE, vectors=VOCAB_VECTORS)
    labels_fieldspec.build_vocab(train_data)
    vocab_size = len(phrases_fieldspec.vocab)
    output_size = len(labels_fieldspec.vocab)
    
    train_iter, eval_iter = data.BucketIterator.splits(
        (train_data, eval_data), 
        batch_size=BATCH_SIZE,
        device=device, sort=False, shuffle=True)
    
    tlog('Data prepared')
    return train_iter, eval_iter, vocab_size, output_size

## The Model

The model below gets to about 62% accuracy on this particular sentiment analysis problem. Below, we describe a few key features of the model.

### Embedding

The first layer of the model is the embedding layer. We described vocabulary embeddings above; here, we show how to take the weights of the pretrained embedding and use them within your own model.

Note that we have set `requires_grad` on the embedding layer's weights to `False` (line 17 below). This freezes the embedding, so that it does not participate in the learning process. Leaving `requires_grad` as `True` for this layer's weights would mean radically increasing the number of learning parameters in the model.

### Recurrent Neural Networks

A *recurrent neural network* RNN may refer to one of two things: The first is a type of neural network layer that maintains a "hidden" state, separate from its learning weights, that lets it carry forward state when it deals with sequential data. The second is a class of neural network layers that includes the basic RNN and more complex constructs employing similar techniques, which include the *gated recurrent unit* (GRU) and *long short-term memory* (LSTM).

For this exercise, we will employ an LSTM. It adds architectural complexity to the model, but has the advantage that it is much less susceptible to the problems of *vanishing gradients* and *exploding gradients* that are often seen when training RNNs on long sequences. We'll use two LSTM layers to start (lines 19-23). One important thing to note about the LSTM is that instead of a single hidden state vector, it maintains a separate *memory cell* with separate mechanisms for determining when to "remember" or "forget" certain computation paths.

### Classification

We'll model our possible integer outputs from 0 to 4 as five possible output classes, and use a linear layer at the end of the model to perform the final classification (line 25).

### The Computation Graph

In the `forward()` method, there are a couple of interesting features to how the model layers are composed.

First, note that the `hidden` state is passed out of the model when the computation returns, and is *passed back in* by the caller on subsequent calls. For the case in which the input hidden state is `None`, the model initializes it correctly.

Second, the `hidden` state consumed and returned by `self.lstm`  is actually a tuple containing the hidden state vector and the memory cell. When we pass the output of the LSTM to the classifier, we specify it as `hidden[0][-1]`. The `[0]` specifies the first element of the (hidden, cell) tuple, and the [-1] specifies the hidden state returned by the final LSTM layer.

In [0]:
# model constants
EMBEDDING_SIZE = 100 # must match dimensions in vocab vectors above
HIDDEN_SIZE = 100
OUTPUT_SIZE = 5 # 0 to 4
NUM_LAYERS = 2



class SentimentAnalyzer(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, pretrained_embedding, hidden_size, output_size):
        super(SentimentAnalyzer, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding.weight.data.copy_(pretrained_embedding)
        self.embedding.weight.requires_grad = False
        
        self.lstm = nn.LSTM(
            input_size=embedding_size,
            num_layers=NUM_LAYERS,
            hidden_size=hidden_size
        )
        
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, phrases, hidden):
        if hidden is None: # tuple w/ 2 for LSTM, make one for RNN or GRU
            hidden = (torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_size, dtype=torch.float).to(device),
                      torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_size, dtype=torch.float).to(device))
        x = self.embedding(phrases)
        x, hidden = self.lstm(x, hidden)
        x = self.fc(hidden[0][-1]) # remove [0] for RNN or GRU, [-1] is for layers
#        return x.squeeze(0), hidden

## Training the Model

The following code cells define some functions to support our model training process, the structure of which should be familiar by now. Go ahead and run them.

In [0]:
def get_model(vocab_size, output_size, vectors):
    tlog('Creating model...')
    sa = SentimentAnalyzer(vocab_size, EMBEDDING_SIZE, vectors, HIDDEN_SIZE, output_size)
    tlog('The model has {} trainable parameters'.format(count_model_params(sa)))
    tlog(sa)
    return sa

In [0]:
def train(model, iterator, loss_fn, optimizer): # one epoch
    curr_loss = 0.
    curr_correct = 0.
    hidden = None
    model.train() # makes sure that training-only fns, like dropout, are active
    
    for batch in iterator:
        # get the data
        phrases, lengths = batch.phrases
        
        if phrases.shape[1] == BATCH_SIZE:        
            # predict and learn
            optimizer.zero_grad()
            guesses, hidden = model(phrases, hidden)
            loss = loss_fn(guesses, batch.labels)
            loss.backward()
            optimizer.step()
            
            hidden[0].detach_() # or we get double-backward errors
            hidden[1].detach_() # or we get double-backward errors

            # measure
            curr_loss += loss.item()
            curr_correct += count_correct(torch.argmax(guesses, 1), batch.labels)
        
    return curr_loss / len(iterator), curr_correct / (len(iterator) * BATCH_SIZE)

In [0]:
def evaluate(model, iterator, loss_fn):
    curr_loss = 0.
    curr_correct = 0.
    hidden = None
    model.eval() # makes sure that training-only fns, like dropout, are inactive
    
    with torch.no_grad(): # not training
        for batch in iterator:
            # get the data
            phrases, lengths = batch.phrases
            
            if phrases.shape[1] == BATCH_SIZE:        
                # predict
                guesses, hidden = model(phrases, hidden) # .squeeze(1)
                loss = loss_fn(guesses, batch.labels)

                # measure
                curr_loss += loss.item()
                curr_correct += count_correct(guesses, batch.labels)

    
    return curr_loss / len(iterator), curr_correct / (len(iterator) * BATCH_SIZE)        

Below, we define the training loop. The basic structures of bookkeeping, training, evaluation, reporting, and saving the model should be familiar.

In [0]:
# training loop constants
EPOCHS = 30
LR = 1e-3


def learn(model, train_iter, eval_iter):
    eval_losses = []
    eval_accs = []
    best_eval_acc = 0
    
    model = model.to(device)

    loss_fn = torch.nn.CrossEntropyLoss()
    # loss_fn = loss_fn.to(device)

    params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(params, lr=LR)
    
    for epoch in range(EPOCHS):
        tlog('EPOCH {} of {}'.format(epoch + 1, EPOCHS))
        
        train_loss, train_acc = train(model, train_iter, loss_fn, optimizer)
        tlog('  Training loss {}   acc {}'.format(train_loss, train_acc))
        
        eval_loss, eval_acc = evaluate(model, eval_iter, loss_fn)
        tlog('  Validation loss {}   acc {}'.format(eval_loss, eval_acc))
        eval_losses.append(eval_loss)
        eval_accs.append(eval_acc)
        if eval_acc > best_eval_acc:
            tlog('  *** New accuracy peak, saving model')
            best_eval_acc = eval_acc
            saved_model_filename = save_model(model, epoch + 1)
    
    tlog('DONE')
    tlog('Best accuracy: {}'.format(best_eval_acc))
    return eval_losses, eval_accs

Let's create our dataset and build the vocabulary:

In [0]:
t_iter, e_iter, vocab_size, output_size = get_data()
embedding_vectors = t_iter.dataset.fields['phrases'].vocab.vectors

And finally, let's create the model and train it:

In [0]:
sa = get_model(vocab_size, output_size, embedding_vectors)
losses, accs = learn(sa, t_iter, e_iter)

In [0]:
for i in range(len(losses)):
    print('{} {}'.format(losses[i], accs[i]))